In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import time

In [11]:
from state_space_setup import *


maximum_parts =  41
state_space = get_state_space(maximum_parts)
state_tuples = list(map(tuple, state_space))

print("Length of the state space generated with max_parts =", maximum_parts, 
      "is:", len(state_space))
print("Index for (2, 3):", get_index(2, 3, maximum_parts))
print("Action space for state (2, 3):", get_action_space(np.array([2, 3]), maximum_parts))

Length of the state space generated with max_parts = 41 is: 903
Index for (2, 3): 86
Action space for state (2, 3): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36]


In [12]:
def transition_probabilities(old_state, new_state, action):
    """
    Input: old_state (np.array): The previous state of the system.
           new_state (np.array): The potential next state of the system.
           action (int): The action taken, representing the number of parts ordered.
    Output: prob (float): The probability of transitioning from old_state to new_state given the action.

    This function calculates the transition probability from one state to another given an action.
    It is important to note that this function assumes the old_state and action are valid to provide
    more general applicability. The transition probabilities are base on three assumputions:
    1. Parts break down as a Poisson process (truncated at 0) with parameter lambda_
    2. The probability of the outstanding parts being delivered is fixed at p for any order size
    3. The newly ordered parts are added to the current outstaning order immediately.
    """
    # Parameters to set
    lambda_ = 2  # Average number of parts breaking down per week
    p = 0.9  # Probability of receiving the ordered parts

    if new_state[1] == 0:
        if (old_state[1] == 0) and (action == 0):
            prob = 1
        else:
            prob = p

        if new_state[0] <= (old_state[0] + old_state[1] + action) and new_state[0] > (old_state[1] + action):
            prob *= stats.poisson.pmf(old_state[0] + old_state[1] + action - new_state[0], mu=lambda_)
        elif new_state[0] == (old_state[1] + action):
            prob *= stats.poisson.sf(old_state[0] - 1, mu=lambda_)
        else:
            prob = 0
    elif new_state[1] == (old_state[1] + action):
        prob = 1 - p
        if new_state[0] <= old_state[0] and new_state[0] > 0:
            prob *= stats.poisson.pmf(old_state[0] - new_state[0],mu=lambda_)
        elif new_state[0] == 0:
            prob *= stats.poisson.sf(old_state[0] - 1, mu=lambda_)
        else:
            prob = 0
    else:
        prob = 0
    return prob

In [13]:
def downtime_cost(num_parts, k=10080, lambda_=2, cost_per_week=16800):
    """
    Input: num_parts (int): The number of parts currently in the inventory.
           k (int): The number of periods in a week (default is 168 hours).
           lambda_ (float): The average number of parts breaking down per week (default is 2).
           cost_per_week (float): The cost incurred if the machine is not running for 
                                  an entire week (default is 16800).
    Output: cost (float): The cost incurred if shortage occurs in the next period for the 
                          given initial inventory level.

    This function calculates the shortage cost for a given state if shortage occurs in 
    the next period. Based on the model assumption - parts breaking as a Poisson process - 
    the shortage cost is calculated so that we get the same expected shortage cost as if we 
    assumed we split the week into k periods and penalise according to the number of periods the 
    machine was out of order.
    """
    if num_parts == 0:
        return cost_per_week

    # vector of numbers of periods of downtime, from 1 to k (e.g., 168 hours a week)
    m = np.arange(1, k + 1)
    # vector of numbers of parts broken before the period in which the last part breaks
    i = np.arange(num_parts)
    # vector of expected number of parts breaking down over periods 1 to k - m for each m
    lambda_m = (lambda_ / k) * (k - m[:, None])  

    # matrix of probabilities for each number of parts breaking down before the period 
    # in which the last part breaks for each such period possible
    pmf = stats.poisson.pmf(i[None, :], lambda_m)
    # vector of probabilities of all the remaining parts breaking down in the next period
    # for each possible number of parts broken before the period in which the last part breaks
    sf = stats.poisson.sf(num_parts - i - 1, lambda_ / k)

    # vector of probabilities of exactly m periods of downtime calculated using the above
    prob = np.sum(pmf * sf, axis=1)
    # weighted cost for each period of downtime, where m/k is the fraction of the week
    weighted_cost = prob * (m / k) * cost_per_week
    # total expected cost as the sum of weighted costs
    total_cost = np.sum(weighted_cost)
    # normalisation by the probability of running out of parts by next week, to obtain
    # the correct expected cost when multiplying by the probability of running out of parts
    normalisation = stats.poisson.sf(num_parts - 1, lambda_)

    return total_cost / normalisation


downtime_vector = np.array([downtime_cost(num_parts) for num_parts in np.arange(maximum_parts + 1)])


def cost_function(old_state, new_state, action):
    """
    Input: old_state (np.array): The previous state of the system.
           action (int): The action taken, representing the number of parts ordered.
           new_state (np.array): The potential next state of the system.
    Output: cost (float): The cost incurred by taking the action in the old state and transitioning to the new state.

    This function calculates the cost incurred by taking an action in a given state and transitioning to a new state.
    The cost is based on the number of parts ordered and the inventory levels before and after the transition.
    """
    global downtime_vector

    holding_cost_as_percentage = 0.008  # percentage of the cost of a part to be paid for holding it in inventory
    ordering_cost = 200  # fixed cost per order placed
    price_per_part = 100  # price per part bought

    # calculate the total cost of the order made
    if action == 0:
        order_cost = 0
    else:
        order_cost = ordering_cost + price_per_part * action

    # calculate the shortage cost if the new state has only the newly arrived parts
    if (new_state[0] == (old_state[1] + action) and new_state[1] == 0) or (new_state[0] == 0):
        shortage_cost = downtime_vector[old_state[0]]
    else:
        shortage_cost = 0

    # calculate holding cost for the new inventory level (paying it for ordered as well does not change the result for the usual parameters)
    holding_cost = holding_cost_as_percentage * price_per_part * new_state[0]

    return holding_cost + order_cost + shortage_cost

In [14]:
# generate the transition probability matrix
trans_prob_matrix = np.zeros((len(state_space), len(state_space), maximum_parts + 1))
# iterate over all old and new states and actions to fill the transition probability matrix
for i, s_old in enumerate(state_space):
    for j, s_new in enumerate(state_space):
        for a in range(maximum_parts + 1):
            trans_prob_matrix[i, j, a] = transition_probabilities(s_old, s_new, a)


# check if the sum of probabilities for each old inventory level and action is 1
all_correct = True
for i, s_old in enumerate(state_space):
    for a in range(maximum_parts - np.sum(s_old) + 1):
        total_prob = round(np.sum(trans_prob_matrix[i, :, a]), 9)
        if total_prob != 1.0:
            print(f"Probability sum for old state {s_old}, and action {a}: {total_prob:.9f}")
            all_correct = False
if all_correct:
    print("All transition probabilities sum to 1 for each old state and action pair. Test passed.")
else:
    print("Some transition probabilities do not sum to 1, see above. Test failed.")


# repeat the same for the cost function
cost_matrix = np.zeros((len(state_space), len(state_space), maximum_parts + 1))
for i, s_old in enumerate(state_space):
    for j, s_new in enumerate(state_space):
        for a in range(maximum_parts + 1):
            cost_matrix[i, j, a] = cost_function(s_old, s_new, a)


# save the transition probability and cost matrices to files
np.save('cost_matrix.npy', cost_matrix)
np.save('trans_prob_matrix.npy', trans_prob_matrix)

All transition probabilities sum to 1 for each old state and action pair. Test passed.


In [15]:
def expected_action_value(current_state, action, values):
    """
    Input: current_state (tuple): The current state of the system.
           action (int): The action taken, representing the number of parts ordered.
           values (np.array): The value function for each state.
    Output: expected_value (float): The expected value of taking the action in the current state.
    """
    # Using global variables for the matrices and the maximum number of parts
    global maximum_parts
    global cost_matrix
    global trans_prob_matrix

    discount_factor = 0.995  # discount factor for future rewards

    # get the index of the current state in the state space
    current_state_idx = get_index(current_state[0], current_state[1], maximum_parts)

    probs = trans_prob_matrix[current_state_idx, :, action]
    costs = cost_matrix[current_state_idx, :, action]
    expected_value = np.dot(probs, costs + discount_factor * values)
    return expected_value


# testing the expected_action_value
current_state = (5, 2)
action = 3

expected_value = expected_action_value(current_state, action, values=np.zeros(len(state_space)))
print(f"Expected value for action {action} in state {current_state}: {expected_value}\n")

current_idx = get_index(current_state[0], current_state[1], maximum_parts)

check_df = pd.DataFrame({
    "Next State": state_tuples,
    "Transition Probability": trans_prob_matrix[current_idx, :, action],
    "Cost": cost_matrix[current_idx, :, action]
})
display(check_df[check_df["Transition Probability"] > 0].sort_values(by="Transition Probability", ascending=False))

Expected value for action 3 in state (5, 2): 694.9610055831398



,Next State,Transition Probability,Cost
308,"(8, 0)",0.243604,506.400000
342,"(9, 0)",0.243604,507.200000
273,"(7, 0)",0.162402,505.600000
375,"(10, 0)",0.121802,508.000000
237,"(6, 0)",0.081201,504.800000
200,"(5, 0)",0.047388,4092.455605
128,"(3, 5)",0.027067,502.400000
167,"(4, 5)",0.027067,503.200000
88,"(2, 5)",0.018045,501.600000
205,"(5, 5)",0.013534,504.000000


In [16]:
# Value iteration
error = 1e-16
values = np.zeros(len(state_space))
delta = 2 * error
counter = 0
elapsed = 0

while delta >= error:
    start_time = time.time()
    delta = 0
    old_values = np.copy(values)
    for i, state in enumerate(state_space):
        v = values[i]
        action_space = get_action_space(state, maximum_parts)
        action_costs = np.array([
            expected_action_value(state, act, old_values)
            for act in action_space
        ])
        values[i] = np.min(action_costs)
        delta = max(delta, abs(v - values[i]))
    counter += 1
    end_time = time.time()
    elapsed += end_time - start_time
    if counter % 400 == 0:
        avg_time = elapsed / 400
        print(f"Finished {counter} iterations \nCurrent delta = {delta}\n")
        print(f"{avg_time:.4f} seconds per iteration")
        print(values)
        print()
        elapsed = 0

print(values)
np.save('values.npy', values)

Finished 400 iterations 
Current delta = 32.87463763082633

0.0714 seconds per iteration
[61262.83076569 61162.83076569 61062.83076569 60962.83076569
 60862.83076569 60762.83076569 60662.83076569 60562.83076569
 60462.83076569 60352.56696614 60238.94964422 60133.39250118
 60021.72186183 59904.41399766 59790.93685772 59678.32995448
 59566.24790807 59454.8030113  59344.04203204 59233.96259078
 59124.55852885 59015.82738678 58907.76774663 58800.37804939
 58693.65661804 58587.60177051 58482.21183843 58377.4851602
 58273.42007799 58170.01493779 58067.26808972 57965.17788805
 57863.74269114 57762.9608615  57662.83076569 57563.35077438
 57464.5192623  57366.33460825 57268.79519509 57171.89940971
 57075.64564303 56980.03229001 53888.95940832 53788.95940832
 53688.95940832 53588.95940832 53488.95940832 53388.95940832
 53288.95940832 53188.95940832 53088.95940832 52976.94918333
 52864.32755867 52758.01519771 52645.64817564 52528.81353194
 52415.44389486 52302.90182871 52190.89849124 52079.538080

In [17]:
state_tuples = list(map(tuple, state_space))

policy_records = []
for state in state_tuples:
    action_space = get_action_space(state, maximum_parts)
    action_costs = np.array([
        expected_action_value(state, act, values)
        for act in action_space
    ])
    order = action_space[np.argmin(action_costs)]
    policy_records.append((state, order))

policy_df = pd.DataFrame(policy_records, columns=["State", "Order_size"])
display(policy_df)
policy_df.to_csv("policy.csv", index=False)

,State,Order_size
0,"(0, 0)",34
1,"(0, 1)",33
2,"(0, 2)",32
3,"(0, 3)",31
4,"(0, 4)",30
...,...,...
898,"(39, 1)",0
899,"(39, 2)",0
900,"(40, 0)",0
901,"(40, 1)",0


In [18]:
policy_df["IP"] = policy_df["State"].apply(sum)
policy_df["New_IP"] = policy_df["IP"] + policy_df["Order_size"]
filtered_policy_df = policy_df[policy_df["Order_size"] != 0]
s = filtered_policy_df["IP"].max()
S = filtered_policy_df["New_IP"].max()
print(f"s = {s}, S = {S}")

s = 11, S = 36


This gives an (s,S) policy with $s = 11$ and $S = 36$ where the current inventory position is the variable of interest. However, for inventory level less than or equal to 3 we get slightly different $s$ and in some cases also $S$. (Both generally lower than the overall)